In [1]:
library(tidyverse)
library(yaml)
library(rstan)
library(tidybayes)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: StanHeaders


rstan version 2.32.3 (Stan version 2.26.1)


For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For within-chain threading using `reduce_sum()` or `map_rect()` Stan functions,
chan

## Config and data

In [2]:
config_id <- "pooling_sim_partial_pool"
data_config_id <- "pooling_sim"

In [3]:
data_config <- yaml.load_file(paste0("../data/configs/", data_config_id, "/data.yaml"))
main_config <- yaml.load_file(paste0("../experiments/configs/", config_id, "/main.yaml"))

In [4]:
results_base_dir <- paste0("../experiments/results/", config_id)
if (!dir.exists(results_base_dir)) {
    dir.create(results_base_dir, recursive = TRUE)
}

### Data

In [5]:
data_base_dir <- paste0("../", data_config$output_dir)
data_path <- paste0(data_base_dir, "/data.csv")
beta_0 <- data_config$beta_0
beta_1 <- data_config$beta_1

In [6]:
data_df <- read_csv(data_path)
fit_data_df <- data_df %>%
    filter(measurement_id < 11)
eval_data_df <- data_df %>%
    filter(measurement_id == 11)

Rows: 4400 Columns: 8
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl (8): group_id, group_effect, x, indiv_effect, indiv_id, measurement_id, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Model

In [7]:
model_seed <- main_config$seed
n_samp <- main_config$n_samp
n_warmup_samp <- main_config$n_warmup_samp
n_chains <- main_config$n_chains
model_file_path <- paste0("../", main_config$model_file_path)

## Model fitting

Note that we are only doing "no pooling" for the intercept. We assume a fixed slope.

In [8]:
stan_data <- list(
    N = eval_data_df %>% nrow(),
    n_groups = eval_data_df %>% distinct(group_id) %>% nrow(),
    y = eval_data_df %>% pull(y),
    x = eval_data_df %>% pull(x),
    group_id = eval_data_df %>% pull(group_id)
)

In [9]:
if (!file.exists(paste0(results_base_dir, "/fit.rds"))) {
    fit <- stan(
        file = model_file_path,
        data = stan_data,
        chains = n_chains,
        iter = n_samp,
        warmup = n_warmup_samp,
        seed = model_seed,
        cores = n_chains,
        pars = c("beta_0", "beta_1"),
        include = TRUE
    )
    saveRDS(fit, paste0(results_base_dir, "/fit.rds"))
    
    tidy_draws_df <- fit %>% tidybayes::spread_draws(beta_0[i], beta_1)
    tidy_draws_df %>%
        write_csv(paste0(results_base_dir, "/draws.csv"))
} else {
    fit <- readRDS(paste0(results_base_dir, "/fit.rds"))
    tidy_draws_df <- read_csv(paste0(results_base_dir, "/draws.csv"))
}

In [12]:
tidy_draws_long_df <- tidy_draws_df %>%
    pivot_longer(cols = !c(".chain", ".iteration", ".draw", "i"), names_to = "param") %>%
    ungroup() %>%
    rename(group_id = i)

In [13]:
fit_summary <- summary(fit)

In [14]:
fit_summary$summary %>%
    as_tibble(rownames = "param")
    # filter(param == "beta_1")

param,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
beta_0[1],162.222243,0.0392246813,1.235543990,159.7780368,161.3821418,162.2349933,163.0808284,164.6173245,992.1962,1.004261
beta_0[2],140.610750,0.0409660580,1.221046974,138.2409278,139.7811590,140.6153878,141.4197768,143.0482006,888.4160,1.003445
beta_0[3],147.658395,0.0410643580,1.231005971,145.2744981,146.8280706,147.6607792,148.5077735,150.0871015,898.6493,1.003341
beta_0[4],169.803988,0.0387980427,1.209541557,167.4759871,168.9813424,169.7893887,170.6250894,172.1606869,971.9008,1.003568
beta_0[5],145.308981,0.0416992614,1.245638178,142.9637786,144.4502589,145.2938022,146.1501718,147.7504396,892.3333,1.003517
beta_0[6],133.333824,0.0390460059,1.181438045,131.0729152,132.5070243,133.3355073,134.1343631,135.6666862,915.5218,1.003874
beta_0[7],156.587411,0.0393087301,1.189655394,154.2584477,155.7897298,156.5965124,157.3938769,158.9202644,915.9343,1.005596
beta_0[8],152.394608,0.0384410836,1.202184783,150.0713940,151.5947389,152.3880594,153.2037265,154.7921093,978.0278,1.004064
beta_0[9],158.425522,0.0416059643,1.273795097,155.8920748,157.5628519,158.4378274,159.2874742,160.9195418,937.3202,1.003793
